## About data set

There are 3 of datasets that we will use for the analysis:

1. order_dataset
order_id
* customer_id
* order_status
* order_purchase_timestamp
* order_approved_at
* order_delivered_carrier_date
* order_delivered_customer_date
* order_estimated_delivery_date

2. customers_dataset
* customer_id
* customer_unique_id
* customer_zip_code_prefix
* customer_city
* customer_state

3. order_payment_dataset
* order_id
* payment_sequential
* payment_type
* payment_installments
* payment_value

Customer segmentation is one of the most important marketing tools, because it can help a business to better understand its target audience. Segmenting customer helps in better understanding the customers and thus personalizing marketing and communication for each segment.

We will segment our customer using RFM analysis

* Recency (R) : Difference between days since last purchase and observed days.
* Frequency (F) : How many of transcaction each customer made?
* Monetary (M) : How much each customer spend ?

In [6]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from sklearn.cluster import KMeans

In [7]:
url_1 = 'https://docs.google.com/spreadsheets/d/1x3LLvzMhsNLNgd-aX-WkI8iD1SU0OS1V/edit#gid=1719877516'
url_sheet_trf_1 = url_1.replace('/edit#gid=', '/export?format=xlsx&gid=')

# load the order_dataset
df_order = pd.read_excel(url_sheet_trf_1)
pd.set_option('display.max.columns',None)
# get the overview of the dataset
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [8]:

#checking info from the dataset
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [9]:
#checking number of columns and row
df_order.shape

(99441, 8)

In [10]:
url_2 = 'https://docs.google.com/spreadsheets/d/1f3OoU0BfN8izUfFWtQD7g71maLN27DsS/edit#gid=39572284'
url_sheet_trf_2= url_2.replace('/edit#gid=', '/export?format=xlsx&gid=')


# load the order_payments_dataset
df_payment = pd.read_excel(url_sheet_trf_2)
pd.set_option('display.max.columns',None)

#get the overview of the dataset
df_payment.head()

HTTPError: HTTP Error 410: Gone

In [ ]:
# checking info from dataset
df_payment.info()
     

In [ ]:
#check the number of columns and rows
df_payment.shape

In [11]:

url_3 = 'https://docs.google.com/spreadsheets/d/1STF0q38BrjAX1ZHPJkV-KR_ywshFfPWB/edit#gid=439007706'
url_sheet_trf_3 = url_3.replace('/edit#gid=', '/export?format=xlsx&gid=')

# load the customers_dataset
df_cust = pd.read_excel(url_sheet_trf_3)
pd.set_option('display.max.columns',None)

# get overview of the dataset
df_cust.head()

HTTPError: HTTP Error 410: Gone

In [12]:
#checking info from the dataset
df_cust.info()

NameError: name 'df_cust' is not defined

## Cleaning the Dataset